In [1]:
import json

import pickle
import numpy as np
import jionlp as jio
import numpy as np
import matplotlib.pyplot as plt

`jio.help()` is provided to search how to use jio functions.


In [2]:
def load_data_json(json_path):
    f = open(json_path, 'r', encoding='utf-8')
    json_data = json.load(f)
    f.close()
    return json_data

In [3]:
import pandas as pd
import glob
# files = glob.glob("./data/Keywords/*.xlsx")
files = glob.glob("./data/Keywords/02pest.list.xlsx")
same_words_dict = {}

for file in files:
    df = pd.read_excel(file, header=None)
    df_list = df.values.tolist()
    for words in df_list:
        change_to = words[0]
        same_words_dict[change_to] = change_to
        for word in words:
            word = str(word)
            if word == 'nan':
                break
            if word != change_to:
                same_words_dict[word] = change_to
                

In [4]:
json_path = f'./data/train_complete.json'
json_data = load_data_json(json_path)

results_path = f'./output_results/train_result_colbert_neg100_epoch1'
with open(results_path, 'rb') as handle:
    results = pickle.load(handle)
            

In [5]:
print(len(results))

560


In [9]:
binary_result_path  = f'./output_results/train_binary_result_cross_encoder_binary_neg60_alldata_epoch2'
with open(binary_result_path, 'rb') as handle:
    binary_results = pickle.load(handle)

In [10]:
print(len(binary_results))

560


In [11]:
print(len(json_data))
print(results['1'][:3])

560
[('3', 0, -44.131023), ('5', 0, -34.71434), ('6', 0, -33.386192)]


In [12]:
def softmax(x):
    """Compute the softmax in a numerically stable way."""
    x = x - np.max(x)
    exp_x = np.exp(x)
    softmax_x = exp_x / np.sum(exp_x)
    return softmax_x

In [13]:
from sklearn.metrics import f1_score, recall_score, precision_score

from sklearn.preprocessing import MultiLabelBinarizer
multi = MultiLabelBinarizer()

all_dids = []
did2data = {}
for d in json_data:
    did = d['did']
    did2data[did] = d
    all_dids.append(did)


In [14]:
did2data['1126']

{'did': '1126',
 'title': '氣候多變溫差大，臺南區農業改良請農友慎防水稻葉稻熱病',
 'text': '受鋒面影響，導致溫度高低起伏，早晚溫差大的不穩定氣候正是水稻稻熱病的好發條件，臺南區農業改良場近期監測已有部分地區發生，特籲請農友注意加強防治，以避免稻熱病蔓延發生，以減少損失，確保收成。一期作水稻葉稻熱病發生適期為3月下旬至5月上旬，尤以4月上、中旬，即清明節過後半個月以內發生最為猖獗，農友應適時做有效的防治。葉稻熱病的病斑可分為抵抗型（褐色）、急性型（綠色）及停止型（灰綠色）三種。抵抗型病斑一般發生於抗病品種或病害流行期前一週；急性型病斑乃天候適合發病時出現的病斑，通常由下位葉開始發生，似綠豆皮色，可迅速蔓延到上位葉；停止型病斑則為急性型病斑遇到發病條件受阻時退化的老病斑，病斑呈紡錘形，且較前二種為大。葉稻熱病的防治方法，在管理上切勿施用過量的氮肥，年初以來氣溫較低，部分地區一期作插秧後生育較慢，農民心急可能會再多施氮肥，當氣溫回升後，過多的氮肥使得植株徒長，葉片柔弱，降低對病害之抗性，將增加稻熱病發生機會，因此農友更應注意防範病害之發生，如發現稻熱病發生，應停施氮肥，並立即引水灌溉，因灌溉水中有多量的矽酸，水稻吸收後可增加對稻熱病的抵抗性，另應減少進入稻田踐踏，以免使稻根受損，影響稻株之抗病力。葉稻熱病的防治藥劑可分為預防用之粒劑如4％或6％撲殺熱粒劑，每公頃每次用量30公斤，於好發病時期前一週施下(約在3月中、下旬)，並保持水深3～5公分，持續4～5天。防治用之噴撒劑如75％三賽唑可濕性粉劑3,000倍或41.7％三賽唑水懸劑1,500倍，於插秧後35～45天撒佈一次，經14天再施藥一次。如已出現急性型病斑可噴施50％護粒松乳劑1,000倍或45％喜樂克拉乳劑1,200倍（加展著劑CS-72000倍）或40%亞賜圃可濕性粉劑1000～1500倍（加展著劑新利農3000倍）或15％加普胺水懸劑2000倍或20％嘉賜三賽唑可濕性粉劑1,500倍，隔7天再施藥一次，共施用2次，其他防治藥劑可參考植物保護手冊選用防治。晚植水稻應注意鉀肥之配合，以增加抗病性。早植稻若田間也發現二化螟蟲幼蟲，可混合二化螟蟲登記藥劑同時防治，如9％培丹粒劑或0.3％芬普尼粒劑或50％撲滅松乳劑1000倍或22.5％陶斯松乳劑450倍等，其他防治藥劑可參考植物

In [15]:
from gensim.summarization import bm25
corpus_ids = [d['did'] for d in json_data]
corpus = []
titles = []
title_and_corpus = []

from zhon.hanzi import punctuation
import string
for i, d in enumerate(json_data):
    text_words = d['replaced_text_sentence'].split()
    title_words = d['replaced_title_sentence'].split()

    filter_title_words = [w for w in title_words if w not in punctuation and w not in string.punctuation]

    filter_text_words = [w for w in text_words if w not in punctuation and w not in string.punctuation]

    corpus.append(filter_text_words)
    titles.append(filter_title_words)
    title_and_corpus.append(filter_title_words + filter_text_words)

bm25Model = bm25.BM25(title_and_corpus)

In [16]:
for i, query in enumerate((title_and_corpus)):
    q_did = corpus_ids[i]
    result = results[q_did]
#     bm25_scores = []
    scores = bm25Model.get_scores(query)
    for idx, score in enumerate(scores):
        ref_did = corpus_ids[idx]
        if q_did == ref_did:
            continue
        for k, r in enumerate(result):
            if r[0] == ref_did:
                r = r + (score,)
                result[k] = r
                break
    results[q_did] = result


In [17]:
# keyword_sets = []
for i, d in enumerate(json_data):
    did = d['did']
    text_words = d['text_sentence'].split()
    title_words = d['title_sentence'].split()
    all_words = text_words + title_words
    keyword_set = set()
    for k, v in same_words_dict.items():
        if k in all_words:
            keyword_set.add(same_words_dict[k])
#     text = ' '.join(all_words)
#     key_phrases = jio.keyphrase.extract_keyphrase(text)
#     for key_phrase in key_phrases:
#         if key_phrase in same_words_dict:
#             keyword_set.add(same_words_dict[key_phrase])
#         else:
#             keyword_set.add(key_phrase)
#     keyword_sets.append(keyword_set)
    json_data[i]['keyword_set'] = keyword_set
    
            

In [18]:
def jaccard_score(set1, set2):
    u = set1.union(set2) 
    i = set1.intersection(set2)
    if len(u) == 0:
        return 0
    return len(i) / len(u)

In [19]:
from collections import defaultdict
keyword_set_sim = defaultdict(dict)
for i in range(len(json_data)):
    for k in range(len(json_data)):
        
        did_i = json_data[i]['did']
        did_k = json_data[k]['did']
        
        set_i = json_data[i]['keyword_set']
        set_k = json_data[k]['keyword_set']
        
        sim_socre = jaccard_score(set_i, set_k)
        keyword_set_sim[did_i][did_k] = sim_socre

In [20]:
for i in range(len(json_data)):
    did_i = json_data[i]['did']
    result = results[did_i]
    
    
    set_sim_score = keyword_set_sim[did_i]
    for ref_did, score in set_sim_score.items():
        if did_i == ref_did:
            continue
        for k, r in enumerate(result):
            if r[0] == ref_did:
                r = r + (score,)
                result[k] = r
                break
    results[did_i] = result

In [21]:
def show_pos_score(pos_dids, scores, all_ref_ids):
    for pos_did in pos_dids:
        pos_did_score = scores[np.where(all_ref_ids==pos_did)[0]]
        print(f'pos_did : {pos_did} softmax score: {pos_did_score}')

In [22]:
def get_static_info(li):
    mean = np.mean(li)
    median = np.median(li)
    std = np.std(li)
    max_ = np.max(li)
    min_ = np.min(li)
    print(f' mean {mean} median {median}, std {std}, max {max_}, min {min_}')


In [95]:
topk = 300
all_pos_scores = []
all_scores = []
c = 0
avg_pos_hit_rate = 0

y = []
y_bm25 = []
y_pred = []
y_sets = []
y_second_pred = []
y_binary = []
y_mixed = []
bm25_avg_length = 0
train_binary_pairs = []
train_pos_count = 0
train_neg_count = 0

second_num_of_std = 1.4
avg_pos_rate = 0.0
count = 0

num_of_std = 1.5
binary_th = 0.7
set_th = 1.0

for k, values in results.items():
    did = str(k)
    
    binary_value = binary_results[k]
    
    data = did2data[did]
    pos_dids = data['pos_dids']
    count += len(pos_dids)
    if len(pos_dids) == 0:
        continue
    
    all_ref_ids = np.array([v[0] for v in values])


    print(f'did = {k}')
    print('label pos dids')
    print(pos_dids)
#     bm25_pos_dids = data['bm25_pos_dids'][:topk]
    
    bm25_scores = [v[3] for v in values]
    bm25_scores = np.array(bm25_scores)
    bm25_mean = np.mean(bm25_scores)
    bm25_median = np.median(bm25_scores)
    bm25_std = np.std(bm25_scores)
    bm25_max_score = np.max(bm25_scores)
    bm25_th = bm25_median + num_of_std * bm25_std
    bm25_pos_idx = bm25_scores >= bm25_th
    bm25_preds = (all_ref_ids[bm25_pos_idx])
    bm25_pos_scores = bm25_scores[bm25_pos_idx]
    
    
    binary_scores = [v[1] for v in binary_value]
    binary_scores = np.array(binary_scores)
    binary_pos_idx = binary_scores >= binary_th
    binary_preds = all_ref_ids[binary_pos_idx]
    binary_pos_scores = binary_scores[binary_pos_idx]
    


    set_scores =  [v[4] for v in values]
    set_scores = np.array(set_scores)
    set_pos_idx = set_scores >= set_th
    set_preds = (all_ref_ids[set_pos_idx])
    set_pred_scores = (set_scores[set_pos_idx])
    
    

    
    scores =  [v[2] for v in values]
    scores = np.array(scores)
    mean = np.mean(scores)
    median = np.median(scores)
    std = np.std(scores)
    max_score = np.max(scores)
    th = median + num_of_std * std
    pos_idx = scores >= th
    preds = all_ref_ids[pos_idx]
    pos_scores = scores[pos_idx]
    
    
    first_scores = bm25_pos_scores
    first_preds = bm25_preds
    
    secnod_pred_mean = np.mean(first_scores)
    second_median = np.median(first_scores)
    second_std = np.std(first_scores)
    second_th = second_median + second_num_of_std * second_std
    second_pos_idx = first_scores >=  second_th
    second_preds = first_preds[second_pos_idx]
    second_pos_scores = first_scores[second_pos_idx]
#     print(second_preds)
#     print(second_pos_scores)
    
#     mix_preds = list(set(bm25_preds) & set(binary_preds))
#     mix_preds2 = list(set(binary_preds) & set(bm25_preds))
    mix_preds = list(set(bm25_preds) | set(set_preds))
    mix_preds = list(set(mix_preds) & set(binary_preds))

    print(mix_preds)

    
#     set_and_binary_scores = set_scores * binary

    
#     mix_scores = alpha * set_scores + (1-alpha) * binary
    
    
#     print(binary_scores.shape)

#     print(f' mean {mean}, std {std}, max {max_score}')
    
#     be_check_scores = binary_scores
#     for pos_did in pos_dids :
#         idx = np.where(all_ref_ids == pos_did)
#         p_score = be_check_scores[idx]
#         print(f'did : {all_ref_ids[idx]}, score : {p_score}')
#         all_pos_scores.append(p_score[0])
#     all_scores += list(be_check_scores)
    
#     scores = softmax(scores)
#     topk_idx = scores.argsort()[::-1][:topk]
    


    
#     print('score based predict topk did and score')
    
#     print(preds)
#     if len(pos_dids) > 0:
#         for pos_did in pos_dids:
#             train_binary_pairs.append((did, pos_did, 1))
#             train_pos_count += 1
#         for pred in preds:
#             if pred in pos_dids:
#                 continue
#             train_binary_pairs.append((did, pred, 0))
#             train_neg_count += 1
#         print(len(pos_dids), len(pred), len(pos_dids)/(len(pred) + len(pos_dids)))

    y.append(pos_dids)
    y_bm25.append(bm25_preds)
    y_pred.append(preds)
    y_second_pred.append(second_preds)
    y_sets.append(set_preds)
    y_binary.append(binary_preds)
    y_mixed.append(mix_preds)



did = 3
label pos dids
['649', '415']
[]
did = 9
label pos dids
['5']
['5']
did = 25
label pos dids
['41', '32']
['32', '41']
did = 26
label pos dids
['37']
['436', '516', '186', '511']
did = 27
label pos dids
['46']
['46']
did = 29
label pos dids
['72']
[]
did = 32
label pos dids
['25', '41']
['41', '25']
did = 34
label pos dids
['40']
[]
did = 35
label pos dids
['191', '75', '66']
['66']
did = 40
label pos dids
['34']
['34']
did = 41
label pos dids
['25', '32']
['32', '25']
did = 44
label pos dids
['1049', '40', '806', '1054', '34']
['40', '1054', '806', '34', '1049']
did = 46
label pos dids
['27']
['27']
did = 49
label pos dids
['498', '95', '790', '505', '759', '46', '375', '344', '27']
['470', '27', '790', '759', '46']
did = 54
label pos dids
['27', '46', '40', '34']
['34']
did = 55
label pos dids
['1049', '838', '858', '482', '556', '480', '805', '806', '1054']
['858', '1054', '806', '482', '906', '1049', '556', '924', '480', '805']
did = 57
label pos dids
['29', '64', '12', '72'

['1263', '1255', '1173', '1228', '1326', '1195', '1289', '1300', '1315', '1310', '1320']
did = 1249
label pos dids
['1276', '1302', '1180', '1172', '1232', '1190', '1230', '1199', '1207', '1298', '1274', '1187', '1198', '1290', '1339']
['1282', '1172', '1230', '1302', '1232', '1180', '1339']
did = 1253
label pos dids
['1324', '1337']
['1324', '1354', '1337']
did = 1255
label pos dids
['1247', '1315', '1289', '1263', '1310', '1326', '1300']
['1263', '1173', '1247', '1326', '1289', '1300', '1315', '1310', '1237', '1320']
did = 1261
label pos dids
['1234', '1276', '1302', '1350', '1305', '1319', '1282', '1298', '1290', '1339', '1323']
['1282', '1249', '1341', '1276', '1290', '1187', '1304', '1319', '1230', '1302', '1232', '1323', '1301', '1305', '1339', '1180', '1298']
did = 1265
label pos dids
['1280', '1292', '1327', '1244', '1311']
[]
did = 1267
label pos dids
['1255', '1247', '1315', '1289', '1263', '1320', '1310', '1173', '1326', '1300']
['1263', '1255', '1173', '1247', '1228', '1326

In [96]:
# avg_pos_softmax_score / c
# 水稻病蟲 , 水稻水象鼻蟲 白葉枯病, 柑桔窄胸天牛
# did2data['612']

In [97]:
# print(len(y))
# print(len(y_bm25))
# print(len(y_pred))

multi = MultiLabelBinarizer()
y_multi = multi.fit_transform(y)
y_bm25_multi = multi.transform(y_bm25)
y_pred_multi = multi.transform(y_pred)
y_second_pred_multi = multi.transform(y_second_pred)
y_set_multi = multi.transform(y_sets)
y_binary_multi = multi.transform(y_binary)
y_mixed_multi = multi.transform(y_mixed)


# print(y_multi.shape)
# print(y_bm25_multi.shape)
# print(y_pred_multi.shape)

average = 'macro'
bm25_f1  = f1_score(y_multi, y_bm25_multi ,average=average)
bm25_recall  = recall_score(y_multi, y_bm25_multi ,average=average)
bm25_precision  = precision_score(y_multi, y_bm25_multi ,average=average)

print('bm25')
print(f'f1:{bm25_f1}, recall:{bm25_recall}, precision:{bm25_precision} ')

pred_f1  = f1_score(y_multi, y_pred_multi ,average=average)
pred_recall  = recall_score(y_multi, y_pred_multi ,average=average)
pred_precision  = precision_score(y_multi, y_pred_multi ,average=average)

print('colbert pred')
print(f'f1:{pred_f1}, recall:{pred_recall}, precision:{pred_precision} ')

second_pred_f1  = f1_score(y_multi, y_second_pred_multi ,average=average)
second_pred_recall  = recall_score(y_multi, y_second_pred_multi ,average=average)
second_pred_precision  = precision_score(y_multi, y_second_pred_multi ,average=average)

print('second pred')
print(f'f1:{second_pred_f1}, recall:{second_pred_recall}, precision:{second_pred_precision} ')


set_f1  = f1_score(y_multi, y_set_multi ,average=average)
set_recall  = recall_score(y_multi, y_set_multi ,average=average)
set_precision  = precision_score(y_multi, y_set_multi ,average=average)

print('set')
print(f'f1:{set_f1}, recall:{set_recall}, precision:{set_precision} ')


binary_f1  = f1_score(y_multi, y_binary_multi ,average=average)
binary_recall  = recall_score(y_multi, y_binary_multi ,average=average)
binary_precision  = precision_score(y_multi, y_binary_multi ,average=average)

print('binary')
print(f'f1:{binary_f1}, recall:{binary_recall}, precision:{binary_precision} ')


mixed_f1  = f1_score(y_multi, y_mixed_multi ,average=average)
mixed_recall  = recall_score(y_multi, y_mixed_multi ,average=average)
mixed_precision  = precision_score(y_multi, y_mixed_multi ,average=average)

print('mixed')
print(f'f1:{mixed_f1}, recall:{mixed_recall}, precision:{mixed_precision} ')

bm25
f1:0.29773886526126314, recall:0.8619092047292412, precision:0.21134240265647267 
colbert pred
f1:0.38122692121042717, recall:0.6371959509472781, precision:0.33553157313475185 
second pred
f1:0.4376728700212978, recall:0.4799138650884264, precision:0.5031037006541329 
set
f1:0.33414442023057883, recall:0.5511156063898054, precision:0.327027724650203 
binary
f1:0.5036091851507282, recall:0.5140584636409085, precision:0.5610037166847642 
mixed
f1:0.4970813648957307, recall:0.484357539912228, precision:0.5925252941103085 


/home/nlplab/harry/anaconda3/lib/python3.8/site-packages/sklearn/preprocessing/_label.py:860: UserWarning: unknown class(es) ['1', '1011', '1016', '105', '1056', '1068', '1082', '1086', '1087', '1093', '110', '1118', '1128', '113', '1130', '1132', '1142', '1146', '1147', '1148', '1151', '1153', '1155', '1157', '1159', '1161', '1165', '117', '1185', '119', '1191', '1206', '121', '1239', '1261', '1271', '1301', '1329', '1332', '1334', '1341', '1352', '1354', '1364', '1365', '1366', '1375', '138', '1390', '14', '140', '1401', '142', '147', '150', '152', '155', '157', '178', '184', '185', '189', '199', '201', '209', '21', '215', '219', '225', '242', '248', '26', '262', '263', '267', '268', '273', '283', '295', '297', '299', '3', '301', '303', '310', '311', '313', '319', '329', '333', '337', '342', '343', '351', '365', '369', '370', '371', '381', '387', '39', '392', '403', '411', '421', '441', '449', '453', '458', '463', '465', '471', '474', '484', '501', '504', '508', '51', '510', '516', '

In [92]:
print('pos score')
get_static_info(all_pos_scores)
print(len(all_pos_scores))
n, bins, patches = plt.hist(all_pos_scores)
plt.show()

print('all score')
get_static_info(all_scores)
print(len(all_scores))

n, bins, patches = plt.hist(all_scores)
plt.show()

pos score


/home/nlplab/harry/anaconda3/lib/python3.8/site-packages/numpy/core/fromnumeric.py:3372: RuntimeWarning: Mean of empty slice.
  return _methods._mean(a, axis=axis, dtype=dtype,
/home/nlplab/harry/anaconda3/lib/python3.8/site-packages/numpy/core/_methods.py:170: RuntimeWarning: invalid value encountered in double_scalars
  ret = ret.dtype.type(ret / rcount)
/home/nlplab/harry/anaconda3/lib/python3.8/site-packages/numpy/core/_methods.py:233: RuntimeWarning: Degrees of freedom <= 0 for slice
  ret = _var(a, axis=axis, dtype=dtype, out=out, ddof=ddof,
/home/nlplab/harry/anaconda3/lib/python3.8/site-packages/numpy/core/_methods.py:194: RuntimeWarning: invalid value encountered in true_divide
  arrmean = um.true_divide(
/home/nlplab/harry/anaconda3/lib/python3.8/site-packages/numpy/core/_methods.py:226: RuntimeWarning: invalid value encountered in double_scalars
  ret = ret.dtype.type(ret / rcount)


ValueError: zero-size array to reduction operation maximum which has no identity

In [ ]:
print(len(train_binary_pairs))
print(train_pos_count, train_neg_count)
print(train_pos_count / len(train_binary_pairs))
train_binary_pairs[:10]

In [ ]:
import pickle
with open('./data/train_binary_pairs', 'wb') as handle:
    pickle.dump(train_binary_pairs, handle, protocol=pickle.HIGHEST_PROTOCOL)